In [5]:
with open('paloozabot.log','r') as file:
    logs = file.readlines() 

In [7]:
line = logs[0]

In [8]:
line

'05/01/2024 03:33:36 PM:WARNING:discord.client:client:295:__init__: PyNaCl is not installed, voice will NOT be supported\n'

In [9]:


#(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [AP]M)
#(\w+)
#(.+?)

In [10]:
type(match)

re.Match

In [14]:
match.groups()

('05/01/2024 03:33:36 PM',
 'WARNING',
 'discord.client',
 'client',
 '295',
 '__init__',
 'PyNaCl is not installed, voice will NOT be supported')

In [17]:
match = re.search(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [AP]M):(\w+):(.+?):(.+?):(\d+):(\w+): (.+)', line)
if match:
    timestamp = datetime.strptime(match.group(1), '%m/%d/%Y %I:%M:%S %p')
    level = match.group(2)
    module = match.group(3)
    context = match.group(4)
    line_number = match.group(5)
    function = match.group(6)
    message = match.group(7)

if 'Message from' in message:
    # Extract message-related fields
    user_match = re.search(r'Message from (\w+)', message)
    channel_match = re.search(r'in #(\w+) \(ID: (\d+)\)', message)
    content_match = re.search(r': (.+), Message ID', message)

    if user_match and channel_match and content_match:
        user_id = user_match.group(1)
        channel_name = channel_match.group(1)
        channel_id = int(channel_match.group(2))
        content = content_match.group(1)

        # Extract mentions and emojis (if any)
        mentions = re.findall(r'<@(\d+)>', content)
        emojis = re.findall(r':\w+:', content)

        # Insert data into the Message table
        insert_message(timestamp, user_id, channel_name, channel_id, content, mentions, emojis)

elif 'invoked' in message:
    # Extract command-related fields
    user_match = re.search(r'(\w+) invoked (\w+) command', message)

    if user_match:
        user_id = user_match.group(1)
        command = user_match.group(2)


In [18]:
import re
from datetime import datetime

def parse_log_file(file_path):
    with open(file_path, 'r') as file:
        log_data = file.readlines()

    for line in log_data:
        # Extract relevant fields using regular expressions
        match = re.search(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2} [AP]M):(\w+):(.+?):(.+?):(\d+):(\w+): (.+)', line)
        if match:
            timestamp = datetime.strptime(match.group(1), '%m/%d/%Y %I:%M:%S %p')
            level = match.group(2)
            module = match.group(3)
            context = match.group(4)
            line_number = match.group(5)
            function = match.group(6)
            message = match.group(7)

            if 'Message from' in message:
                # Extract message-related fields
                user_match = re.search(r'Message from (\w+)', message)
                channel_match = re.search(r'in #(\w+) \(ID: (\d+)\)', message)
                content_match = re.search(r': (.+), Message ID', message)

                if user_match and channel_match and content_match:
                    user_id = user_match.group(1)
                    channel_name = channel_match.group(1)
                    channel_id = int(channel_match.group(2))
                    content = content_match.group(1)

                    # Extract mentions and emojis (if any)
                    mentions = re.findall(r'<@(\d+)>', content)
                    emojis = re.findall(r':\w+:', content)

                    # Insert data into the Message table
                    insert_message(timestamp, user_id, channel_name, channel_id, content, mentions, emojis)

            elif 'invoked' in message:
                # Extract command-related fields
                user_match = re.search(r'(\w+) invoked (\w+) command', message)

                if user_match:
                    user_id = user_match.group(1)
                    command = user_match.group(2)

                    # Insert data into the Command table
                    insert_command(timestamp, user_id, command)

            else:
                # Insert data into the System table
                insert_system(timestamp, level, module, context, message)

def insert_message(timestamp, user_id, channel_name, channel_id, content, mentions, emojis):
    # Insert data into the Message table
    print(f"Inserting into Message table: {timestamp}, {user_id}, {channel_name}, {channel_id}, {content}, {mentions}, {emojis}")

def insert_command(timestamp, user_id, command):
    # Insert data into the Command table
    print(f"Inserting into Command table: {timestamp}, {user_id}, {command}")

def insert_system(timestamp, level, module, context, message):
    # Insert data into the System table
    print(f"Inserting into System table: {timestamp}, {level}, {module}, {context}, {message}")

# Usage example
# parse_log_file('logfile.log')

In [19]:
parse_log_file('paloozabot.log')

Inserting into System table: 2024-05-01 15:33:36, WARNING, discord.client, client, PyNaCl is not installed, voice will NOT be supported
Inserting into System table: 2024-05-01 15:33:36, DEBUG, discord.client, client, on_ready has successfully been registered as an event
Inserting into System table: 2024-05-01 15:33:36, DEBUG, asyncio, selector_events, Using selector: EpollSelector
Inserting into System table: 2024-05-01 15:33:36, INFO, discord.client, client, logging in using static token
Inserting into System table: 2024-05-01 15:33:37, INFO, discord.gateway, gateway, Shard ID None has connected to Gateway (Session ID: 8d0f6a8f9f354eda4c4327c900d5c823).
Inserting into System table: 2024-05-01 15:33:39, INFO, discord, bot, Logged in as paloozabot
Inserting into Command table: 2024-05-01 15:34:40, a_na4603, hello
Inserting into System table: 2024-05-01 18:34:57, INFO, discord.gateway, gateway, Shard ID None has successfully RESUMED session 8d0f6a8f9f354eda4c4327c900d5c823.
Inserting int